# Adjust info

This notebook is used to consider late entries, reschedule matches, and move players from and into subteams.

In [1]:
from helper_functions import setup
import numpy as np
import matplotlib.pyplot as plt
import helper_functions as hf
from helper_functions.setup.team_creation import find_best_team_to_join
from helper_functions.setup.openai_image_download import generate_all_images, save_resized_animal_images


## Register latecomers

Add all latecoming players, and add them as a reserve in each subteam for the best-fitting team.

In [2]:
player_df = setup.sanitize_and_anonymize_data(overwrite=True, verbose=False)
late = player_df[player_df["late_entry"]]
teams = hf.get_teams()
subteams = hf.ALL_SUBTEAMS.values()

for _, player in late.iterrows():
    name = player["nickname"]
    # Don't do anything if the player has already been added
    if any([team.contains_player(name) for team in teams]):
        continue
    team_to_join = teams[find_best_team_to_join(teams, player)]
    team_to_join.add_player(player, register_as_reserve=True)
    print(f"{name} joins {team_to_join.name}")
    generate_all_images([name])
    save_resized_animal_images(150)


## Move players from reserve to main team

If players are attending a sport, even if just in reserve, and are substituted in, they should be registered there.

In [6]:
sport = "running_sprints"
player_name = "Pushy Bulldog"

def move_player_from_reserve_to_subteam(player_name: str, new_subteam: int | str, sport: str, save=True):
    """Move a player from the reserve subteam in the given sport to the subteam with the provided key."""
    player_df = hf.get_players().set_index("nickname", drop=False)
    current_subteam = player_df.loc[player_name, f"subteam_{sport}"]
    player_team: str = player_df.loc[player_name, "Team"]  # type: ignore
    team_letter = player_team.replace("Team ", "")
    assert f"{sport}_{team_letter}: {new_subteam}" in hf.ALL_SUBTEAMS
    if current_subteam != "R":
        print(f"The player you've chosen ({player_name}) is currently not in a reserve team for {sport}, skipping.")
        return
    print(f"Setting new subteam '{new_subteam}' in {sport} for {player_name}")
    player_df.loc[player_name, f"subteam_{sport}"] = new_subteam
    if save:
        fpath = hf.Team.backup_path_from_letter(team_letter)
        player_df[player_df["Team"] == player_team].to_csv(fpath, index=False)
move_player_from_reserve_to_subteam("Magnificent Barracuda", 2, "volleyball")


The player you've chosen (Magnificent Barracuda) is currently not in a reserve team for volleyball, skipping.


## Reschedule Matches

Some matches need to be rescheduled. We can do that here.

In [27]:
from datetime import datetime, time
from helper_functions.setup.match_scheduling import write_match_backup_from_df
from itertools import combinations

def reschedule_match_to_new_time(match_key: str, new_time: datetime | time, check_overlaps: bool = False):
    matches = hf.get_match_df().set_index("full_key", drop=False)
    old_time = matches.loc[match_key, "start"]
    if isinstance(new_time, time):
        new_time = datetime.combine(old_time, new_time)  # type: ignore
        print(new_time)
    matches.loc[match_key, "start"] = new_time
    matches.loc[match_key, "day"] = new_time.strftime("%A")
    matches.loc[match_key, "time"] = new_time.strftime("%H:%M")
    print(f"Rewritten the time for {match_key} to {new_time.strftime("%H:%M (%A)")}.")
    write_match_backup_from_df(matches, True)
    if check_overlaps:
        matches = hf.get_matches()
        num_conflicts = np.sum([comb[0].has_hard_collision(comb[1], verbose=True) for comb in combinations(matches, 2)])
        if num_conflicts == 0:
            print("No conflicts found.")

reschedule_match_to_new_time("basketballA: 1B: 1", time(17, 45))
reschedule_match_to_new_time("basketballB: 1C: 1", time(18, 30))
reschedule_match_to_new_time("basketballC: 1A: 1", time(19, 15), True)


2024-04-29 17:45:00
Rewritten the time for basketballA: 1B: 1 to 17:45 (Monday).
2024-04-29 18:30:00
Rewritten the time for basketballB: 1C: 1 to 18:30 (Monday).
2024-04-29 19:15:00
Rewritten the time for basketballC: 1A: 1 to 19:15 (Monday).
No conflicts found.


## Changing match locations

For some matches, it might make sense to move them around if they are back-to-back, but currently scheduled at different courts.

In [44]:

def change_match_location(match_key: str, new_time: datetime | time, check_overlaps: bool = False):
    matches = hf.get_match_df().set_index("full_key", drop=False)
    return
    write_match_backup_from_df(matches, True)
    if check_overlaps:
        matches = hf.get_matches()
        num_conflicts = np.sum([comb[0].has_hard_collision(comb[1], verbose=True) for comb in combinations(matches, 2)])
        if num_conflicts == 0:
            print("No conflicts found.")

matches = hf.get_matches()
# Filter for all combinations that happen back-to-back:
back_to_back = [comb for comb in combinations(matches, 2) if (comb[0].end == comb[1].start) and (len(set(comb[0].involved_players).intersection(comb[1].involved_players))>0)]
# filter away sports where this isn't relevant
back_to_back = [comb for comb in back_to_back if comb[0].sport not in ["basketball", "chess", "tennis", "football", "capture_the_flag"]]
# Filter away combinations where the locations already match:
back_to_back = [comb for comb in back_to_back if comb[0].location != comb[1].location]
match_keys = [str((comb[0].match_key, comb[0].location, comb[1].match_key, comb[1].location)) for comb in back_to_back]
print("\n".join(match_keys))
match_df = hf.get_match_df()
match_df[match_df["sport"] == "spikeball"]


('spikeball_A: 4_B: 3', '1', 'spikeball_B: 3_C: 1', '2')
('spikeball_B: 3_C: 1', '2', 'spikeball_C: 1_A: 1', '3')
('beer_pong_A: 3_B: 2', '1', 'beer_pong_B: 2_C: 1', '2')
('beer_pong_A: 1_B: 3', '1', 'beer_pong_B: 3_C: 2', '2')
('beer_pong_B: 1_C: 3', '1', 'beer_pong_C: 3_A: 2', '2')
('ping_pong_A: 06_B: 14', '3', 'ping_pong_C: 06_A: 06', '1')
('ping_pong_B: 01_C: 13', '1', 'ping_pong_C: 13_A: 09', '2')
('ping_pong_B: 01_C: 13', '1', 'ping_pong_A: 13_B: 01', '3')
('ping_pong_B: 16_C: 08', '1', 'ping_pong_A: 14_B: 16', '3')
('ping_pong_A: 13_B: 01', '3', 'ping_pong_C: 16_A: 13', '2')
('ping_pong_A: 03_B: 12', '1', 'ping_pong_B: 12_C: 04', '3')


,sport,team_a,team_b,location,day,time,result,winner,start,duration,team_a_key,team_b_key,full_key.1,full_key
33,spikeball,A: 3,B: 2,2,Thursday,18:15,NaN,NaN,2024-05-02 18:15:00,1800,A: 3,B: 2,spikeballA: 3B: 2,spikeballA: 3B: 2
34,spikeball,A: 2,B: 1,3,Thursday,19:45,NaN,NaN,2024-05-02 19:45:00,1800,A: 2,B: 1,spikeballA: 2B: 1,spikeballA: 2B: 1
35,spikeball,A: 1,B: 4,1,Thursday,19:45,NaN,NaN,2024-05-02 19:45:00,1800,A: 1,B: 4,spikeballA: 1B: 4,spikeballA: 1B: 4
36,spikeball,A: 4,B: 3,1,Thursday,18:15,NaN,NaN,2024-05-02 18:15:00,1800,A: 4,B: 3,spikeballA: 4B: 3,spikeballA: 4B: 3
37,spikeball,B: 2,C: 4,1,Thursday,17:45,NaN,NaN,2024-05-02 17:45:00,1800,B: 2,C: 4,spikeballB: 2C: 4,spikeballB: 2C: 4
38,spikeball,B: 1,C: 2,1,Thursday,18:45,NaN,NaN,2024-05-02 18:45:00,1800,B: 1,C: 2,spikeballB: 1C: 2,spikeballB: 1C: 2
39,spikeball,B: 4,C: 3,2,Thursday,17:45,NaN,NaN,2024-05-02 17:45:00,1800,B: 4,C: 3,spikeballB: 4C: 3,spikeballB: 4C: 3
40,spikeball,B: 3,C: 1,2,Thursday,18:45,NaN,NaN,2024-05-02 18:45:00,1800,B: 3,C: 1,spikeballB: 3C: 1,spikeballB: 3C: 1
41,spikeball,C: 4,A: 4,2,Thursday,19:45,NaN,NaN,2024-05-02 19:45:00,1800,C: 4,A: 4,spikeballC: 4A: 4,spikeballC: 4A: 4
42,spikeball,C: 2,A: 3,1,Thursday,19:15,NaN,NaN,2024-05-02 19:15:00,1800,C: 2,A: 3,spikeballC: 2A: 3,spikeballC: 2A: 3
